# TODO

- [x] IDENTIFICAÇÃO DAS CIDADES LITORÂNEAS
    - [ ] CONSIDERAR CIDADES ESPECÍFICAS
- [x] LEVANTAMENTO DA QUANTIDADE DE HOSPITAIS
    - [x] FILTRAR HOSPITAIS QUE POSSUEM UTI NEONATAL
- [x] LEVANTAMENTO DA SEGURANÇA
    - [x] TAXAS DE HOMICIDIO
    - [x] TAXAS DE ASSALTOS
    - [x] TAXAS DE CRIMES VIOLENTOS
- [x] LEVANTAMENTO DE EDUCAÇÃO
    - [x] INDICE DAS ESCOLAS NO PAÍS
    - [x] QUANTIDADE DE ESCOLAS NA CIDADE
- [ ] LEVANTAMENTO SOBRE LAZER
    - [ ] QTD DE PARQUES
    - [x] QTD DE SHOPPINGS
    - [x] TOTAL DE ÁREAS VERDES
    - [x] QTD DE PRAIAS
- [ ] LEVANTAMENTO DA MÉDIA DE ALUGUÉIS  
- [x] LEVANTAMENTO DOS PREÇOS DE ALIMENTO
- [ ] DEFINIÇÃO DE CUSTO BENEFÍCIO

## CIDADES A CONSIDERAR

- SP
    - BERTIOGA
    - CANANEIA
    - CARAGUATATUBA
    - GUARUJA
    - IGUAPE
    - ILHABELA
    - ILHA COMPRIDA
    - ITANHAEM
    - MONGAGUA
    - PERUIBE
    - PRAIA GRANDE
    - SANTOS
    - SAO SEBASTIAO
    - SAO VICENTE
    - UBATUBA

- ES
    - VITORIA
    - VILA VELHA

- PB
    - JOAO PESSOA

- SC
    - BALNEARIO CAMBORIU
    - FLORIANOPOLIS

- PR
    - CURITIBA

# IMPORTAÇÃO DE BIBLIOTECAS

In [1]:
import polars as pl
import os
import duckdb

In [2]:
AUXILIAR_PATH = r'finding-home/finding_home/auxiliar/'
DATA_PATH = r'finding-home/finding_home/data/'

In [3]:
con = duckdb.connect(os.path.join(DATA_PATH, "DW.db"))

# IMPORTAÇÃO DE DADOS

## CIDADES

In [4]:
from unicodedata import normalize 
def custnorm(In_series):
    for i, x in enumerate(In_series):
        newvalue = normalize('NFKD',x).encode('ascii', errors='ignore').decode('utf-8')
        if newvalue != x:
            In_series[i]=newvalue
    return In_series

In [4]:
# cities = pl.read_csv(os.path.join(DATA_PATH, 'CIDADES_LITORANEAS_IBGE.csv'), separator=';') \
#            .filter(pl.col('NM_REGIAO').is_in(['Sul', 'Sudeste', 'Nordeste'])) \
#            .with_columns(pl.col('NM_MUN').str.to_uppercase().alias('MUNICIPIO')) \
#            .with_columns(pl.col('MUNICIPIO').map_batches(custnorm).alias('MUNICIPIO'))

In [5]:
cities = pl.read_csv(os.path.join(DATA_PATH, 'cidades_ibge_full.csv'), separator='|') \
            .select(pl.col('COD_IBGE'), pl.col('UF'), pl.col('MUNICIPIO'), pl.col('POPULACAO')) \
            .with_columns(pl.col('MUNICIPIO').alias('CIDADE')) \
            .with_columns(pl.col('MUNICIPIO').str.to_uppercase().alias('MUNICIPIO')) \
           .with_columns(pl.col('MUNICIPIO').map_batches(custnorm).alias('MUNICIPIO'))

In [6]:
con.execute("""CREATE OR REPLACE TABLE cities AS SELECT * FROM cities ORDER BY 'UF', 'MUNICIPIO'""")

## HOSPITAIS

In [7]:
hospitals = pl.read_csv(os.path.join(DATA_PATH, 'hospitais.csv'), separator='|', encoding='cp1252', truncate_ragged_lines=True).filter(pl.col('UTI_NEONATAL_EXIST') > 0)

In [8]:
con.sql("""
    CREATE OR REPLACE TABLE hospitals_stat AS
    SELECT 
            REGIAO
        ,   UF
        ,   MUNICIPIO
        ,   SUM(CASE WHEN DESC_NATUREZA_JURIDICA = 'HOSPITAL_PRIVADO' AND DS_TIPO_UNIDADE == 'HOSPITAL GERAL' THEN 1 ELSE 0 END) AS HPRI_HOSPITAL_GERAL
        ,   SUM(CASE WHEN DESC_NATUREZA_JURIDICA = 'HOSPITAL_PRIVADO' AND DS_TIPO_UNIDADE == 'PRONTO SOCORRO GERAL' THEN 1 ELSE 0 END) AS HPRI_PRONTO_SOCORRO_GERAL 
        ,   SUM(CASE WHEN DESC_NATUREZA_JURIDICA = 'HOSPITAL_PRIVADO' AND DS_TIPO_UNIDADE == 'UNIDADE MISTA' THEN 1 ELSE 0 END) AS HPRI_UNIDADE_MISTA
        ,   SUM(CASE WHEN DESC_NATUREZA_JURIDICA = 'HOSPITAL_PRIVADO' AND DS_TIPO_UNIDADE == 'PRONTO SOCORRO ESPECIALIZADO' THEN 1 ELSE 0 END) AS HPRI_PRONTO_SOCORRO_ESPECIALIZADO
        ,   SUM(CASE WHEN DESC_NATUREZA_JURIDICA = 'HOSPITAL_PRIVADO' AND DS_TIPO_UNIDADE == 'HOSPITAL ESPECIALIZADO' THEN 1 ELSE 0 END) AS HPRI_HOSPITAL_ESPECIALIZADO
        ,   SUM(CASE WHEN DESC_NATUREZA_JURIDICA = 'HOSPITAL_PRIVADO' THEN 1 ELSE 0 END) AS QTD_HOSPITAL_PRI
    
        ,   SUM(CASE WHEN DESC_NATUREZA_JURIDICA = 'HOSPITAL_PÚBLICO' AND DS_TIPO_UNIDADE == 'HOSPITAL GERAL' THEN 1 ELSE 0 END) AS HPUB_HOSPITAL_GERAL
        ,   SUM(CASE WHEN DESC_NATUREZA_JURIDICA = 'HOSPITAL_PÚBLICO' AND DS_TIPO_UNIDADE == 'PRONTO SOCORRO GERAL' THEN 1 ELSE 0 END) AS HPUB_PRONTO_SOCORRO_GERAL 
        ,   SUM(CASE WHEN DESC_NATUREZA_JURIDICA = 'HOSPITAL_PÚBLICO' AND DS_TIPO_UNIDADE == 'UNIDADE MISTA' THEN 1 ELSE 0 END) AS HPUB_UNIDADE_MISTA
        ,   SUM(CASE WHEN DESC_NATUREZA_JURIDICA = 'HOSPITAL_PÚBLICO' AND DS_TIPO_UNIDADE == 'PRONTO SOCORRO ESPECIALIZADO' THEN 1 ELSE 0 END) AS HPUB_PRONTO_SOCORRO_ESPECIALIZADO
        ,   SUM(CASE WHEN DESC_NATUREZA_JURIDICA = 'HOSPITAL_PÚBLICO' AND DS_TIPO_UNIDADE == 'HOSPITAL ESPECIALIZADO' THEN 1 ELSE 0 END) AS HPUB_HOSPITAL_ESPECIALIZADO
        ,   SUM(CASE WHEN DESC_NATUREZA_JURIDICA = 'HOSPITAL_PÚBLICO' THEN 1 ELSE 0 END) AS QTD_HOSPITAL_PUB
    
        ,  COUNT(DESC_NATUREZA_JURIDICA) AS QTD_HOSPITAIS
    
    FROM hospitals

    GROUP BY REGIAO
        ,   UF
        ,   MUNICIPIO

    ORDER BY REGIAO
        ,   UF
        ,   MUNICIPIO

""")

In [9]:
con.sql("""SELECT * FROM hospitals_stat WHERE MUNICIPIO = 'OSASCO'""").to_df()

,REGIAO,UF,MUNICIPIO,HPRI_HOSPITAL_GERAL,HPRI_PRONTO_SOCORRO_GERAL,HPRI_UNIDADE_MISTA,HPRI_PRONTO_SOCORRO_ESPECIALIZADO,HPRI_HOSPITAL_ESPECIALIZADO,QTD_HOSPITAL_PRI,HPUB_HOSPITAL_GERAL,HPUB_PRONTO_SOCORRO_GERAL,HPUB_UNIDADE_MISTA,HPUB_PRONTO_SOCORRO_ESPECIALIZADO,HPUB_HOSPITAL_ESPECIALIZADO,QTD_HOSPITAL_PUB,QTD_HOSPITAIS
0,SUDESTE,SP,OSASCO,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,3


# ALIMENTOS

In [10]:
food_filter = [
                'ARROZ LONGO FINO BENEFICIADO TIPO 1 (5 kg)'
                , 'FEIJÃO COMUM CORES TIPO 1 (kg)'
                , 'FEIJÃO COMUM PRETO TIPO 1 (kg)'
                , 'CARNE BOVINA ACÉM (kg)'
                , 'CARNE BOVINA COSTELA (kg)'
                , 'CARNE BOVINA PONTA DE AGULHA (kg)'
                , 'CARNE DE FRANGO CONGELADO (kg)'
                , 'CARNE DE FRANGO RESFRIADO (kg)'
                , 'CARNE SUÍNA CONGELADO PERNIL COM OSSO (kg)'
                , 'CARNE SUÍNA RESFRIADO PERNIL COM OSSO (kg)'
                , 'OLEO DE SOJA REFINADO (900 ml)'
                , 'LEITE DE VACA LONGA VIDA INTEGRAL (l)'
                , 'MACARRÃO ESPAGUETE SÊMOLA COM OVOS (500 g)'
                , 'MACARRÃO ESPAGUETE SÊMOLA COM OVOS (kg)'
                , 'SAL (kg)'
                ]

In [11]:
food = pl.read_csv(os.path.join(DATA_PATH, 'ALIMENTOS_CONAB.csv'), separator=';') \
         .melt(id_vars=['UF', 'PRODUTO'], value_vars=pl.selectors.numeric(), variable_name='MES_REF', value_name='PRECO') \
         .filter(pl.col('PRODUTO').is_in(food_filter)) \
         .pivot(values='PRECO', columns='PRODUTO', index='UF', aggregate_function='mean') \
         .fill_null(0.0)
         

In [12]:
con.execute("""CREATE OR REPLACE TABLE food AS SELECT * FROM food ORDER BY 'UF'""")

# SEGURANÇA

## MORTES

In [13]:
homicidios = pl.read_csv(os.path.join(DATA_PATH, 'taxa-homicidios.csv'), separator=';') \
                  .filter(pl.col('período') > 2016) \
                  .with_columns(pl.col('nome').str.to_uppercase().alias('MUNICIPIO')) \
                  .with_columns(pl.col('MUNICIPIO').map_batches(custnorm).alias('MUNICIPIO')) \
                  .unique() \
                  .group_by(['cod', 'MUNICIPIO']).agg(pl.mean('valor').alias('MEDIA_HOMICIDIO_3Y')) \
                  .rename({'cod': 'COD_IBGE'})

In [14]:
con.execute("""CREATE OR REPLACE TABLE homicidios AS SELECT * FROM homicidios ORDER BY 'COD_IBGE'""")

## MORTES ARMA

In [15]:
hom_arma = pl.read_csv(os.path.join(DATA_PATH, 'taxa-de-homicidios-por-armas-de-fogo.csv'), separator=';') \
                  .filter(pl.col('período') > 2016) \
                  .with_columns(pl.col('nome').str.to_uppercase().alias('MUNICIPIO')) \
                  .with_columns(pl.col('MUNICIPIO').map_batches(custnorm).alias('MUNICIPIO')) \
                  .unique() \
                  .group_by(['cod', 'MUNICIPIO']).agg(pl.mean('valor').alias('MEDIA_HOM_ARMA_3Y')) \
                  .rename({'cod': 'COD_IBGE'})

In [16]:
con.execute("""CREATE OR REPLACE TABLE hom_arma AS SELECT * FROM hom_arma ORDER BY 'COD_IBGE'""")

## MORTES VIOLENTAS

In [17]:
m_violentas = pl.read_csv(os.path.join(DATA_PATH, 'mortes-violentas.csv'), separator=';') \
                  .filter(pl.col('período') > 2016) \
                  .with_columns(pl.col('nome').str.to_uppercase().alias('MUNICIPIO')) \
                  .with_columns(pl.col('MUNICIPIO').map_batches(custnorm).alias('MUNICIPIO')) \
                  .unique() \
                  .group_by(['cod', 'MUNICIPIO']).agg(pl.mean('valor').alias('MEDIA_HOM_VIOLENTA_3Y')) \
                  .rename({'cod': 'COD_IBGE'})

In [18]:
con.execute("""CREATE OR REPLACE TABLE m_violentas AS SELECT * FROM m_violentas ORDER BY 'COD_IBGE'""")

## FURTO E ROUBO DE VEÍCULOS

In [19]:
furto_veiculos = pl.read_csv(os.path.join(DATA_PATH, 'furto_roubo_veiculos.csv'), separator=';') \
                    .select(pl.col('UF'), pl.col('ano'), pl.col('qtd'), pl.col('taxa_100mil_veic')) \
                    .filter(pl.col('ano') > 2016) \
                    .unique() \
                    .group_by(['UF']).agg(pl.mean('taxa_100mil_veic').alias('MEDIA_FURTO_VEICULO_3Y')) 

In [20]:
con.execute("""CREATE OR REPLACE TABLE furto_veiculos AS SELECT * FROM furto_veiculos ORDER BY 'UF'""")

## FURTO E ROUBO DE CELULARES

In [22]:
furto_celulares = pl.read_csv(os.path.join(DATA_PATH, 'furto_roubo_celular.csv'), separator=';') \
                    .select(pl.col('UF'), pl.col('ano'), pl.col('qtd'), pl.col('taxa_100mil_hab')) \
                    .filter(pl.col('ano') > 2016) \
                    .unique() \
                    .group_by(['UF']).agg(pl.mean('taxa_100mil_hab').alias('MEDIA_FURTO_CELULAR_3Y')) 

In [23]:
con.execute("""CREATE OR REPLACE TABLE furto_celulares AS SELECT * FROM furto_celulares ORDER BY 'UF'""")

## ESTUPRO

In [24]:
estupro = pl.read_csv(os.path.join(DATA_PATH, 'estupro.csv'), separator=';') \
                    .select(pl.col('UF'), pl.col('ano'), pl.col('qtd'), pl.col('taxa_100mil_hab')) \
                    .filter(pl.col('ano') > 2016) \
                    .unique() \
                    .group_by(['UF']).agg(pl.mean('taxa_100mil_hab').alias('MEDIA_ABUSOS_3Y')) 

In [25]:
con.execute("""CREATE OR REPLACE TABLE estupro AS SELECT * FROM estupro ORDER BY 'UF'""")

# PRAIAS*

In [26]:
praias = pl.read_csv(os.path.join(DATA_PATH, 'praias.csv'), separator='|', truncate_ragged_lines=True) \
                    .unique() 

In [27]:
con.execute("""CREATE OR REPLACE TABLE praias AS SELECT * FROM praias ORDER BY 'uf', 'cidade'""")

In [33]:
praias.head()

uf,cidade,praia,ponto,latlong,2016,2017,2018,2019,2020,2021,LAT,LONG
str,str,str,str,str,f64,i64,i64,i64,i64,i64,f64,f64
"""AL""","""Barra de Santo…","""Barra de Santo…",null,"""-9.440222, -35…",3.0,3,3,3,-1,3,-9.440222,-35.518556
"""AL""","""Barra de São M…","""Praia do Gunga…","""Estuário da La…","""-9.860111, -35…",3.0,3,3,3,-1,3,-9.860111,-35.905306
"""AL""","""Coruripe""","""Pontal do Coru…",null,"""-10.161306, -3…",3.0,3,3,3,-1,3,-10.161306,-36.13525
"""AL""","""Japaratinga""","""Japaratinga""","""Av. Principal""","""-9.094278, -35…",3.0,2,1,3,-1,2,-9.094278,-35.258472
"""AL""","""Maceió""","""Praia de Jacar…",null,"""-9.60725, -35.…",2.0,2,2,3,-1,1,-9.60725,-35.688056


# ESCOLAS

In [28]:
escolas_qtd = pl.read_csv(os.path.join(DATA_PATH, 'qtd_escolas.csv'), separator=';', truncate_ragged_lines=True) \
                    .unique() 

In [29]:
con.execute("""CREATE OR REPLACE TABLE escolas_qtd AS SELECT * FROM escolas_qtd ORDER BY 'UF', 'COD_MUN'""")

## ENSINO FUNDAMENTAL 1 - 4

In [31]:
escola_1_4 = pl.read_csv(os.path.join(DATA_PATH, 'escola_1-4.csv'), separator=';', truncate_ragged_lines=True) \
                    .unique() 

In [32]:
con.execute("""CREATE OR REPLACE TABLE escolas_fund_i AS 
            SELECT 
                COD_MUN AS COD_IBGE
           ,    UF
           ,    NM_MUN AS MUNICIPIO
           ,    AVG(CASE WHEN REDE = 'Estadual' THEN NOTA ELSE 0 END) NOTA_ESTADUAL
           ,    AVG(CASE WHEN REDE = 'Federal' THEN NOTA ELSE 0 END) NOTA_FEDERAL
           ,    AVG(CASE WHEN REDE = 'Pública' THEN NOTA ELSE 0 END) NOTA_PUBLICA

           FROM 
            escola_1_4
           
           GROUP BY
                COD_MUN
           ,    UF
           ,    NM_MUN
""")

## ENSINO FUNDAMENTAL 5 - 8

In [33]:
escola_5_8 = pl.read_csv(os.path.join(DATA_PATH, 'escola_5-8.csv'), separator=';', truncate_ragged_lines=True) \
                    .unique() 

In [34]:
con.execute("""CREATE OR REPLACE TABLE escolas_fund_ii AS 
            SELECT 
                COD_MUN AS COD_IBGE
           ,    UF
           ,    NM_MUN AS MUNICIPIO
           ,    AVG(CASE WHEN REDE = 'Estadual' THEN NOTA ELSE 0 END) NOTA_ESTADUAL
           ,    AVG(CASE WHEN REDE = 'Federal' THEN NOTA ELSE 0 END) NOTA_FEDERAL
           ,    AVG(CASE WHEN REDE = 'Pública' THEN NOTA ELSE 0 END) NOTA_PUBLICA

           FROM 
            escola_5_8
           
           GROUP BY
                COD_MUN
           ,    UF
           ,    NM_MUN
""")

In [35]:
escolas_fund_ii = duckdb.sql("""
            SELECT 
                COD_MUN AS COD_IBGE
           ,    UF
           ,    NM_MUN AS MUNICIPIO
           ,    AVG(CASE WHEN REDE = 'Estadual' THEN NOTA ELSE 0 END) NOTA_ESTADUAL
           ,    AVG(CASE WHEN REDE = 'Federal' THEN NOTA ELSE 0 END) NOTA_FEDERAL
           ,    AVG(CASE WHEN REDE = 'Pública' THEN NOTA ELSE 0 END) NOTA_PUBLICA

           FROM 
            escola_5_8
           
           GROUP BY
                COD_MUN
           ,    UF
           ,    NM_MUN

           """)

## ENSINO MÉDIO

In [36]:
escola_medio = pl.read_csv(os.path.join(DATA_PATH, 'escola-medio.csv'), separator=';', truncate_ragged_lines=True) \
                    .unique() 

In [39]:
con.execute("""
            CREATE OR REPLACE TABLE escola_medio AS 
            SELECT 
                COD_MUN AS COD_IBGE
           ,    UF
           ,    NM_MUN AS MUNICIPIO
           ,    AVG(CASE WHEN REDE = 'Estadual' THEN NOTA ELSE 0 END) NOTA_ESTADUAL
           ,    AVG(CASE WHEN REDE = 'Federal' THEN NOTA ELSE 0 END) NOTA_FEDERAL
           ,    AVG(CASE WHEN REDE = 'Pública' THEN NOTA ELSE 0 END) NOTA_PUBLICA

           FROM 
            escola_medio
           
           GROUP BY
                COD_MUN
           ,    UF
           ,    NM_MUN
            
""")

# SHOPPINGS

In [40]:
shopping = pl.read_csv(os.path.join(DATA_PATH, 'shoppings_qtd.csv'), separator=';', truncate_ragged_lines=True) \
                    .unique() 

In [41]:
con.execute("""CREATE OR REPLACE TABLE shopping AS 
            SELECT * FROM shopping ORDER BY 'UF'
""")

# ARBORISMO

In [42]:
arborismo = pl.read_csv(os.path.join(DATA_PATH, 'arborismo.csv'), separator=';', truncate_ragged_lines=True) \
                    .unique() 

In [43]:
con.execute("""CREATE OR REPLACE TABLE arborisado AS 
        SELECT 
           COD_IBGE
        ,  UF
        ,  MUNICIPIO
           
        , SUM(CASE WHEN TIPO = 'ARBORIZADO' THEN VALOR ELSE 0 END) / SUM(VALOR) AS PERC_ARBORIZADO
        , SUM(CASE WHEN TIPO = 'NÃO ARBORIZADO' THEN VALOR ELSE 0 END) / SUM(VALOR) AS PERC_N_ARBORIZADO
           
        
        FROM arborismo
        GROUP BY
            COD_IBGE
        ,   UF
        ,   MUNICIPIO
""")

# UNIÃO CIDADES & HOSPITAIS

In [48]:
con.execute("""
        CREATE OR REPLACE VIEW city_info AS
            SELECT 
                CT.COD_IBGE,
                CT.UF,
                CT.MUNICIPIO,
                CT.POPULACAO,
                       
                HS.REGIAO,
                HS.HPRI_HOSPITAL_GERAL,
                HS.HPRI_PRONTO_SOCORRO_GERAL,
                HS.HPRI_UNIDADE_MISTA,
                HS.HPRI_PRONTO_SOCORRO_ESPECIALIZADO,
                HS.HPRI_HOSPITAL_ESPECIALIZADO,
                HS.QTD_HOSPITAL_PRI,
                HS.HPUB_HOSPITAL_GERAL,
                HS.HPUB_PRONTO_SOCORRO_GERAL,
                HS.HPUB_UNIDADE_MISTA,
                HS.HPUB_PRONTO_SOCORRO_ESPECIALIZADO,
                HS.HPUB_HOSPITAL_ESPECIALIZADO,
                HS.QTD_HOSPITAL_PUB,
                HS.QTD_HOSPITAIS,

                "ARROZ LONGO FINO BENEFICIADO TIPO 1 (5 kg)" AS ARROZ_BRANCO_5KG,
                "CARNE BOVINA ACÉM (kg)" AS ACEM,
                "CARNE BOVINA COSTELA (kg)" AS COSTELA,
                "CARNE BOVINA PONTA DE AGULHA (kg)" AS PONTA_DE_AGULHA,
                "CARNE DE FRANGO CONGELADO (kg)" AS FRANGO_CONGELADO,
                "CARNE DE FRANGO RESFRIADO (kg)" AS FRANGO_RESFRIADO,
                "CARNE SUÍNA CONGELADO PERNIL COM OSSO (kg)" AS PERNIL_CONGELADO,
                "CARNE SUÍNA RESFRIADO PERNIL COM OSSO (kg)" AS PERNIL_RESFRIADO,
                "FEIJÃO COMUM CORES TIPO 1 (kg)" AS FEIJAO_CORES,
                "FEIJÃO COMUM PRETO TIPO 1 (kg)" AS FEIJAO_PRETO,
                "LEITE DE VACA LONGA VIDA INTEGRAL (l)" AS LEITE_INTEGRAL,
                ("MACARRÃO ESPAGUETE SÊMOLA COM OVOS (500 g)" + "MACARRÃO ESPAGUETE SÊMOLA COM OVOS (kg)") AS ESPAGUETE,
                "OLEO DE SOJA REFINADO (900 ml)" AS OLEO_REFINADO,
                "SAL (kg)" AS SAL,
               
                TR.PERC_ARBORIZADO,
                TR.PERC_N_ARBORIZADO,
                
                SH.QTD AS QTD_SHOPPINGS_UF,
                
                FI.NOTA_ESTADUAL AS FI_NOTA_ESTADUAL,
                FII.NOTA_ESTADUAL AS FII_NOTA_ESTADUAL,
                EM.NOTA_ESTADUAL AS EM_NOTA_ESTADUAL,

                FI.NOTA_FEDERAL AS FI_NOTA_FEDERAL,
                FII.NOTA_FEDERAL AS FII_NOTA_FEDERAL,
                EM.NOTA_FEDERAL AS EM_NOTA_FEDERAL,

                FI.NOTA_PUBLICA AS FI_NOTA_PUBLICA,
                FII.NOTA_PUBLICA AS FII_NOTA_PUBLICA,
                EM.NOTA_PUBLICA AS EM_NOTA_PUBLICA,
                       
                ES.ESTADUAL AS QTD_ESC_ESTADUAL,
                ES.FEDERAL AS QTD_ESC_FEDERAL,
                ES.MUNICIPAL AS QTD_ESC_MUNICIPAL,
                ES.PRIVADA AS QTD_ESC_PRIVADA,
                       
                AB.MEDIA_ABUSOS_3Y,
                FCL.MEDIA_FURTO_CELULAR_3Y,
                FCR.MEDIA_FURTO_VEICULO_3Y,
               

           FROM cities AS CT

           LEFT JOIN  hospitals_stat AS HS
           ON HS.UF = CT.UF
           AND HS.MUNICIPIO = CT.MUNICIPIO

           LEFT JOIN food FD
           ON FD.UF = CT.UF
                       
           LEFT JOIN arborisado TR
           ON TR.COD_IBGE = CT.COD_IBGE
                       
           LEFT JOIN shopping SH
           ON SH.UF = CT.UF
                    
          LEFT JOIN escola_medio FI
           ON FI.COD_IBGE = CT.COD_IBGE

           LEFT JOIN escolas_fund_ii FII
           ON FII.COD_IBGE = CT.COD_IBGE

           LEFT JOIN escolas_fund_i EM
           ON EM.COD_IBGE = CT.COD_IBGE
                       
           LEFT JOIN escolas_qtd ES
           ON ES.COD_MUN = CT.COD_IBGE
                       
           LEFT JOIN estupro AB
           ON AB.UF = CT.UF
                       
           LEFT JOIN furto_celulares FCL
           ON FCL.UF = CT.UF
          
           LEFT JOIN furto_veiculos FCR
           ON FCR.UF = CT.UF

""")

con.commit()

In [45]:
con.execute('SELECT * FROM city_info LIMIT 4').df()

,COD_IBGE,UF,MUNICIPIO,POPULACAO,REGIAO,HPRI_HOSPITAL_GERAL,HPRI_PRONTO_SOCORRO_GERAL,HPRI_UNIDADE_MISTA,HPRI_PRONTO_SOCORRO_ESPECIALIZADO,HPRI_HOSPITAL_ESPECIALIZADO,...,FI_NOTA_PUBLICA,FII_NOTA_PUBLICA,EM_NOTA_PUBLICA,QTD_ESC_ESTADUAL,QTD_ESC_FEDERAL,QTD_ESC_MUNICIPAL,QTD_ESC_PRIVADA,MEDIA_ABUSOS_3Y,MEDIA_FURTO_CELULAR_3Y,MEDIA_FURTO_VEICULO_3Y
0,2108603,MA,PINHEIRO,84160,NORDESTE,0.0,0.0,0.0,0.0,0.0,...,1.133333,1.466667,1.633333,9,1,123,8,20.900197,333.716656,369.480041
1,2111300,MA,SAO LUIS,1115932,NORDESTE,2.0,0.0,0.0,0.0,0.0,...,1.333333,1.100000,1.225000,102,4,117,317,20.900197,333.716656,369.480041
2,2412203,RN,SAO JOSE DE MIPIBU,44566,NORDESTE,0.0,0.0,0.0,0.0,0.0,...,0.000000,1.066667,1.400000,7,0,20,5,12.149612,564.911422,521.014718
3,2933307,BA,VITORIA DA CONQUISTA,343643,NORDESTE,2.0,0.0,0.0,0.0,0.0,...,1.100000,1.733333,1.866667,17,1,125,63,21.962632,468.161315,405.329131


In [46]:
con.sql("""SELECT * 
        FROM city_info 
        WHERE COD_IBGE LIKE '3205200' """).df()

,COD_IBGE,UF,MUNICIPIO,POPULACAO,REGIAO,HPRI_HOSPITAL_GERAL,HPRI_PRONTO_SOCORRO_GERAL,HPRI_UNIDADE_MISTA,HPRI_PRONTO_SOCORRO_ESPECIALIZADO,HPRI_HOSPITAL_ESPECIALIZADO,...,FI_NOTA_PUBLICA,FII_NOTA_PUBLICA,EM_NOTA_PUBLICA,QTD_ESC_ESTADUAL,QTD_ESC_FEDERAL,QTD_ESC_MUNICIPAL,QTD_ESC_PRIVADA,MEDIA_ABUSOS_3Y,MEDIA_FURTO_CELULAR_3Y,MEDIA_FURTO_VEICULO_3Y
0,3205200,ES,VILA VELHA,508655,SUDESTE,2.0,0.0,0.0,0.0,0.0,...,1.5,1.6,1.933333,22,1,69,52,32.990422,659.153666,492.600863


In [49]:
con.close()

# LINKS AUXILIARES

- [API IBGE CIDADES](https://servicodados.ibge.gov.br/api/v3/agregados/6579/periodos/2021/variaveis/9324?localidades=N6[1100015])
- [API IBGE AGREGADA](https://servicodados.ibge.gov.br/api/docs/agregados?versao=3)
- [ALIMENTOS CONAB](https://sisdep.conab.gov.br/precosiagroweb/)
- [ATLAS DA VIOLENCIA](https://www.ipea.gov.br/atlasviolencia/filtros-series)
- [BASE DOS DADOS](https://basedosdados.org/)
- [FORUM SEGURANÇA](http://forumseguranca.org.br:3838/)
- [ANÁLISE PRAIAS](https://www1.folha.uol.com.br/cotidiano/2023/12/veja-a-qualidade-das-praias-do-litoral-do-brasil.shtml)


In [3]:
import folium
import geopandas as gpd
from folium.plugins import MiniMap
import duckdb
import os

DATA_PATH = r'finding-home/finding_home/data/'

In [4]:
municipios = gpd.read_file(os.path.join(DATA_PATH, "BR_Municipios_2022.zip"))

In [3]:
municipios.geometry = municipios.simplify(200)

In [17]:
municipios[municipios["CD_MUN"] == '4300406'].explore()

In [9]:
municipios.loc[municipios.NM_MUN.str.upper() == 'CUIABÁ'].explore()

In [4]:
state_entry = [f for f in municipios.iterfeatures() if f["properties"]["NM_MUN"].upper() == 'OSASCO']

In [15]:
feature = next(municipios.iterfeatures())
feature['properties']

{'CD_MUN': '1100015',
 'NM_MUN': "Alta Floresta D'Oeste",
 'SIGLA_UF': 'RO',
 'AREA_KM2': 7067.127}

In [16]:
mapa = folium.Map(location=[-14.235004, -51.92528], zoom_start=4)

In [14]:
folium.Choropleth(
    geo_data=municipios.loc[municipios.NM_MUN == 'Osasco'],
    fill_opacity=0.3,
    line_weight=2,
).add_to(mapa)

In [15]:
popup = folium.GeoJsonPopup(fields=["NM_MUN"])

In [10]:
folium.GeoJson(
    municipios.loc[municipios.NM_MUN.str.upper() == 'OSASCO'],
    zoom_on_click=True,
    highlight_function=lambda feature: {
        "fillColor": (
            "green" if "e" in feature["properties"]["NM_MUN"].lower() else "#ffff00"
        ),
    },
    popup=popup,
    popup_keep_highlighted=True,
).add_to(mapa)

In [11]:
MiniMap(toggle_display=True).add_to(mapa)

In [8]:
mapa.fit_bounds(municipios[municipios["NM_MUN"].str.contains('Osasco')]["geometry"].bounds)